In [1]:
import gradio as gr
from langdetect import detect
from googletrans import Translator

# Supported language codes and names
LANGUAGES = {
    "en": "English",
    "ar": "Arabic",
    "es": "Spanish",
    "de": "German",
    "it": "Italian",
    "ro": "Romanian",
    "hi": "Hindi"
}

translator = Translator()

async def translate_text(input_text, source_lang, target_lang):
    if source_lang == "Detect":
        source_lang = detect(input_text)
    
    result = await translator.translate(input_text, src=source_lang, dest=target_lang)
    return result.text


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🌍 Multilingual Translator with Voice (EN, AR, ES, DE, IT, RO, HI)")

    with gr.Row():
        source_lang = gr.Dropdown(["Detect"] + list(LANGUAGES.keys()), value="Detect", label="Source Language")
        target_lang = gr.Dropdown(list(LANGUAGES.keys()), value="en", label="Target Language")

    with gr.Row():
        input_text = gr.Textbox(lines=3, placeholder="Enter or speak text...", label="Input Text")
        mic = gr.Microphone(label="🎤 Voice Input (Optional)", sources=['microphone'], type='filepath')

    
    output_text = gr.Textbox(label="Translated Text")
    tts_output = gr.Audio(label="🔊 Translated Voice Output")

    async def process(mic_file, text_input, src_lang, tgt_lang):
        import speech_recognition as sr
        import tempfile
        text = text_input

        if mic_file:
            r = sr.Recognizer()
            with sr.AudioFile(mic_file) as source:
                audio = r.record(source)
            try:
                text = r.recognize_google(audio)
            except:
                text = "[Voice not recognized]"

        translated = await translate_text(text, src_lang, tgt_lang)

        # Use gTTS for voice output
        from gtts import gTTS
        import os
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
            tts = gTTS(translated, lang=tgt_lang)
            tts.save(f.name)
            audio_path = f.name
        
        return translated, audio_path


    translate_button = gr.Button("🔁 Translate")

    translate_button.click(fn=process,
                           inputs=[mic, input_text, source_lang, target_lang],
                           outputs=[output_text, tts_output])

demo.launch()


c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
